In [ ]:
# bob.refresh {"cron": "15 9-23 * * 1-5", "timeout": 400000}

In [ ]:
import bob
BHC = bob.HealthCheck()
BHC_KEY = "bfc41c24-d278-4bc8-8d95-767e5da44f59"
START = BHC.start(BHC_KEY)
print(START)

In [ ]:
%run __init__.ipynb

In [ ]:
# -- Data source
# ACTUAL
YAHOO_PATH = os.path.join('..', '..', '..', 'prod', 'yahoo-finance')
DB_YAHOO = pd.read_csv(os.path.join(YAHOO_PATH,
                                    'data_output',
                                    'DB_ALL.csv'),
                       sep=";",
                       low_memory=False).drop('Unnamed: 0', axis=1)
if not DB_YAHOO:
    DB_YAHOO = (bob.mongo.
                read_df('DB_ALL', 'yahoo').
                drop('_id', axis=1).
                sort_values(by='DATE_ORDER', ascending=False))

# Get only the last 360 days
DB_ALL = pd.DataFrame()
TICKERS = DB_YAHOO['TICKER'].drop_duplicates().values.tolist()
for ticker in TICKERS:
    tmp_df = DB_YAHOO[DB_YAHOO['TICKER'] == ticker]
    all_dates = (tmp_df['DATE'].
                 drop_duplicates().
                 sort_values(ascending=False).
                 values.
                 tolist())[:360]
    tmp_df = tmp_df[tmp_df['DATE'].isin(all_dates)]
    DB_ALL = DB_ALL.append(tmp_df)

# ML
DB_ALL_ML = pd.read_csv(os.path.join(YAHOO_PATH,
                                     'data_output',
                                     'DB_ALL_ML.csv'),
                        sep=";",
                        low_memory=False).drop('Unnamed: 0', axis=1)
if not DB_ALL_ML:
    DB_ALL_ML = (bob.mongo.
                 read_df('DB_ALL_ML', 'yahoo').
                 drop('_id', axis=1).
                 sort_values(by='DATE_ORDER', ascending=False))

In [ ]:
# -- Transform yahoo finance data DB_ALL (VALUE, VAR, VARP)
def get_all(dataf, df_ml, filter1=None):
    """
    Transform yahoo finance data DB_ALL (VALUE, VAR, VARP)
    """
    # -- ACUTAL DATA
    # Filter data
    if filter1 is None:
        domain = dataf.reset_index(drop=True)
    else:
        domain = dataf[dataf['TYPE'] == filter1]

    domain_ac = pd.DataFrame({'ENTITY': 'WORLDWIDE',
                              'SCENARIO': (pd.
                                           to_datetime(domain['DATE'],
                                                       format='%Y-%m-%d').
                                           dt.strftime('%d/%m/%Y')),
                              'DATE_ORDER': domain['DATE_ORDER'],
                              'LABEL': domain['LABEL'],
                              'GROUPS': 'ACTUAL',
                              'SETTING': "w/o ML",
                              'TYPE': domain['TYPE'],
                              'VALUE': domain['CLOSE'],
                              'VAR': domain['VARV'],
                              'VARP': domain['VARP'],
                              'VOLUME': domain['VOLUME'].astype('int'),
                              'META': domain['META'],
                              'UNIT_VALUE': domain['UNIT'],
                              'UNIT_VARP': ' %'})

    # -- ML DATA
    # Filter data
    if filter1 is None:
        domain = df_ml.reset_index(drop=True)
    else:
        domain = df_ml[df_ml['TYPE'] == filter1].reset_index(drop=True)

    domain_ml = pd.DataFrame({'ENTITY': 'WORLDWIDE',
                              'SCENARIO': (pd.
                                           to_datetime(domain['DATE'],
                                                       format='%Y-%m-%d').
                                           dt.strftime('%d/%m/%Y')),
                              'DATE_ORDER': domain['DATE_ORDER'],
                              'LABEL': domain['LABEL'],
                              'GROUPS': domain['ML'],
                              'SETTING': "with ML",
                              'TYPE': domain['TYPE'],
                              'VALUE': domain['VALUE'],
                              'VAR': domain['VARV'],
                              'VARP': domain['VARP'],
                              'META': domain['META'],
                              'UNIT_VALUE': domain['UNIT'],
                              'UNIT_VARP': ' %'})

    # Prepare to save
    domain = pd.concat([domain_ac, domain_ml], axis=0).reset_index(drop=True)
    domain['LAST_UPDATE'] = datetime.now().strftime('%d/%m/%Y %H:%M:%S')
    domain = (domain.
              sort_values(by=['DATE_ORDER',
                              'ENTITY',
                              'TYPE'],
                          ascending=False).
              reset_index(drop=True))
    return domain

In [ ]:
def save_df_all(domain, domain_num):
    """
    Save to mongo
    """
    domain.to_csv(os.path.join(OUTPUT_FOLDER, domain_num + '.csv'), sep=';')
    bob.mongo.save_df(domain, domain_num, DB_APP, True)
    df_collection = MongoDB[DB_APP][domain_num]
    df_collection.create_index([("SCENARIO", DESCENDING)])
    df_collection.create_index([("LABEL", ASCENDING)])
    return domain

In [ ]:
# -- Economy Highlighted KPIS
DOMAIN200 = get_all(DB_ALL, DB_ALL_ML)
DOMAIN200 = DOMAIN200[DOMAIN200['GROUPS'] == 'ACTUAL']
DOMAIN200 = save_df_all(DOMAIN200, '200')
# DOMAIN200

In [ ]:
# -- Indice Performance
DOMAIN201BIS = get_all(DB_ALL, DB_ALL_ML, 'INDICES').drop(['SCENARIO', 'TYPE'],
                                                          axis=1)
DOMAIN201BIS = save_df_all(DOMAIN201BIS, '201BIS')
# DOMAIN201BIS

In [ ]:
# -- Major Currencies
DOMAIN202BIS = get_all(DB_ALL, DB_ALL_ML, 'CURRENCIES').drop(['ENTITY',
                                                              'SCENARIO',
                                                              'TYPE'],
                                                             axis=1)
DOMAIN202BIS = save_df_all(DOMAIN202BIS, '202BIS')
# DOMAIN202BIS

In [ ]:
# Stories - Market winners / market losers
def domain_203bis(dataf, domain_num):
    """
    Stories - Market winners / market losers
    """
    # -- Get market winners and market losers
    dataf = dataf[~dataf['UNIT_VALUE'].isin([' KRW', ' CNY'])]
    limit = 7
    domain = pd.DataFrame()
    all_dates = (dataf[['DATE_ORDER', 'GROUPS']].
                 sort_values(by=['DATE_ORDER'], ascending=False).
                 drop_duplicates().
                 values.tolist())
    for each_date in all_dates:
        dat = each_date[0]
        groups = each_date[1]
        # Filter on date
        tmp_df = dataf[(dataf['DATE_ORDER'] == dat) &
                       (dataf['GROUPS'] == groups)]

        # Get winner, sort var. in % descending
        df_winner = (tmp_df[(tmp_df['VARP'] > 0) &
                            (tmp_df['VALUE'] > 1)].
                     sort_values(by='VARP', ascending=False))[:limit]
        df_winner['PACKS'] = 'MARKET WINNERS'

        # Get losers, sort var. in % ascending
        df_loser = (tmp_df[(tmp_df['VARP'] < 0) &
                           (tmp_df['VALUE'] > 1)].
                    sort_values(by='VARP', ascending=True))[:limit]
        df_loser['PACKS'] = 'MARKET LOSERS'

        # Append domain
        domain = domain.append(df_winner).append(df_loser)

    # -- Create DF for Toucan
    # 'ENTITY': 'WORLDWIDE',
    domain = pd.DataFrame({'DATE_ORDER': domain['DATE_ORDER'],
                           'LABEL': domain['LABEL'],
                           'GROUPS': domain['GROUPS'],
                           'SETTING': domain['SETTING'],
                           'PACKS': domain['PACKS'],
                           'VALUE': domain['VALUE'],
                           'VAR': domain['VAR'],
                           'VARP': domain['VARP'],
                           'UNIT_VALUE': domain['UNIT_VALUE'],
                           'UNIT_VARP': domain['UNIT_VARP'],
                           'LAST_UPDATE': (datetime.now().
                                           strftime('%d/%m/%Y %H:%M:%S'))})

    # Save dataframe
    domain.to_csv(os.path.join(OUTPUT_FOLDER, domain_num + '.csv'), sep=';')
    bob.mongo.save_df(domain, domain_num, DB_APP, True)
    df_collection = MongoDB[DB_APP][domain_num]
    df_collection.create_index([("DATE_ORDER", DESCENDING),
                                ("PACKS", DESCENDING)])
    return domain


DOMAIN203ALLBIS = get_all(DB_ALL, DB_ALL_ML, 'EQUITIES')
DOMAIN203BIS = domain_203bis(DOMAIN203ALLBIS, '203BIS')
# DOMAIN203BIS

In [ ]:
# -- Commodities
# Data source
def domain_204bis(dataf, domain_num):
    """
    Commodities
    """
    # Create df for all categories
    dataf['PACKS'] = dataf['META']

    df_all = dataf.copy()
    df_all['META'] = ' All categories'
    domain = pd.concat([dataf, df_all])

    # Save to mongo
    domain.to_csv(os.path.join(OUTPUT_FOLDER, domain_num + '.csv'), sep=';')
    bob.mongo.save_df(domain, domain_num, DB_APP, True)
    df_collection = MongoDB[DB_APP][domain_num]
    df_collection.create_index([("SCENARIO", DESCENDING)])
    df_collection.create_index([("LABEL", ASCENDING)])
    df_collection.create_index([("META", ASCENDING)])
    return domain


DOMAIN204BIS = get_all(DB_ALL,
                       DB_ALL_ML,
                       'COMMODITIES').drop(['ENTITY',
                                            'SCENARIO',
                                            'TYPE'],
                                           axis=1)
DOMAIN204BIS = domain_204bis(DOMAIN204BIS, '204BIS')
# DOMAIN204BIS

In [ ]:
def max_date(dataf, domain_num):
    """
    doc string
    """
    # get max date for each ticker
    dataf = dataf[dataf['GROUPS'] == 'ACTUAL']
    domain = pd.DataFrame()

    tickers = (dataf['LABEL'].
               drop_duplicates().
               sort_values(ascending=True).
               values.tolist())
    # print(tickers)
    for ticker in tickers:
        # print(ticker)
        tmp_df = dataf[dataf['LABEL'] == ticker]
        max_df = tmp_df['DATE_ORDER'].max()
        # print(max_df)
        tmp_df = tmp_df[tmp_df['DATE_ORDER'] == max_df]
        domain = domain.append(tmp_df)

    # Save to mongo
    domain.to_csv(os.path.join(OUTPUT_FOLDER, domain_num + '.csv'), sep=';')
    bob.mongo.save_df(domain, domain_num, DB_APP, True)
    df_collection = MongoDB[DB_APP][domain_num]
    df_collection.create_index([("SCENARIO", DESCENDING)])
    df_collection.create_index([("LABEL", ASCENDING)])
    return domain

In [ ]:
# -- Indice Performance (Widget)
DDOMAIN021 = max_date(DOMAIN201BIS, '021')
# DOMAIN021

In [ ]:
# -- Major Currencies
DOMAIN022 = max_date(DOMAIN202BIS, '022')
# DOMAIN022

In [ ]:
# -- Equities
DOMAIN023 = max_date(DOMAIN203ALLBIS, '023')
# DOMAIN023

In [ ]:
# -- Commodities
DOMAIN024 = max_date(DOMAIN204BIS, '024')
# DOMAIN024

In [ ]:
# -- Data source
# ACTUAL
YAHOO_PATH = os.path.join('..', '..', '..', 'prod', 'yahoo-finance')
DB_TREND = pd.read_csv(os.path.join(YAHOO_PATH,
                                    'data_output',
                                    'DB_TREND.csv'),
                       sep=";",
                       low_memory=False).drop('Unnamed: 0', axis=1)
if not DB_TREND == 0:
    DB_TREND = bob.mongo.read_df('DB_TREND', 'yahoo').drop('_id', axis=1)

# ML
DB_TREND_ML = pd.read_csv(os.path.join(YAHOO_PATH,
                                       'data_output'
                                       'DB_TREND_ML.csv'),
                          sep=";",
                          low_memory=False).drop('Unnamed: 0', axis=1)
if not DB_TREND_ML:
    DB_TREND_ML = bob.mongo.read_df('DB_TREND_ML', 'yahoo').drop('_id', axis=1)

In [ ]:
# -- Transform yahoo finance data DB_TREND
def get_trend(dataf, df_ml, variable):
    """
    Transform yahoo finance data DB_TREND
    """
    domain = dataf.copy()[dataf['TYPE'] == variable]
    domain_ac = pd.DataFrame({'ENTITY': 'WORLDWIDE',
                              'DATE': domain['DATE'],
                              'DATE_ORDER': domain['DATE_ORDER'],
                              'LABEL': domain['LABEL'],
                              'GROUPS': 'ACTUAL',
                              'BOTTOM_FILTER': domain['DATE_SCENARIO'],
                              'UPPER_FILTER_M': domain['METRIC'],
                              'UPPER_FILTER_R': "w/o ML",
                              'VALUE': domain['VALUE'],
                              'UNIT_VALUE': domain['UNIT'],
                              'PRECISION': domain['PRECISION'],
                              'OPEN': domain['OPEN'],
                              'HIGH': domain['HIGH'],
                              'LOW': domain['LOW'],
                              'CLOSE': domain['CLOSE'],
                              'VOLUME': domain['VOLUME'],
                              'META': domain['META'],
                              'META1': domain['META1'],
                              'META2': domain['META2']})

    domain = df_ml.copy()[(df_ml['TYPE'] == variable) &
                          (df_ml['ML'] != 'ACTUAL')]
    domain_ml = pd.DataFrame({'ENTITY': 'WORLDWIDE',
                              'DATE': domain['DATE'],
                              'DATE_ORDER': domain['DATE_ORDER'],
                              'LABEL': domain['LABEL'],
                              'GROUPS':  domain['ML'],
                              'BOTTOM_FILTER': domain['DATE_SCENARIO'],
                              'UPPER_FILTER_M': domain['METRIC'],
                              'UPPER_FILTER_R': "with ML",
                              'VALUE': domain['VALUE'],
                              'UNIT_VALUE': domain['UNIT'],
                              'PRECISION': domain['PRECISION'],
                              'META': domain['META'],
                              'META1': domain['META1'],
                              'META2': domain['META2']})

    domain_ml2 = domain_ac.copy()
    domain_ml2['UPPER_FILTER_R'] = "with ML"

    domain = pd.concat([domain_ac,
                        domain_ml,
                        domain_ml2], axis=0).reset_index(drop=True)
    domain['LAST_UPDATE'] = datetime.now().strftime('%d/%m/%Y %H:%M:%S')
    return domain

In [ ]:
def save_df(domain, domain_num):
    """
    Save dataframe
    """
    domain.to_csv(os.path.join(OUTPUT_FOLDER, domain_num + '.csv'), sep=';')
    bob.mongo.save_df(domain, domain_num, DB_APP, True)
    df_collection = MongoDB[DB_APP][domain_num]
    df_collection.create_index([("LABEL", ASCENDING),
                                ("BOTTOM_FILTER", DESCENDING),
                                ("UPPER_FILTER_M", DESCENDING),
                                ("UPPER_FILTER_R", DESCENDING)])
    return domain

In [ ]:
# -- Economy - Indice
DOMAIN201 = save_df(get_trend(DB_TREND, DB_TREND_ML, 'INDICES'), '201')
# DOMAIN201

In [ ]:
# -- Economy - Currencies
DOMAIN202 = save_df(get_trend(DB_TREND, DB_TREND_ML, 'CURRENCIES'), '202')
# DOMAIN202

UPPER_FILTER_202 = (DOMAIN202[['META', 'LABEL']].
                    drop_duplicates().
                    sort_values(by=['META', 'LABEL'], ascending=True).
                    reset_index(drop=True))
bob.mongo.save_df(UPPER_FILTER_202, '202_UPPER_FILTER', DB_APP, True)

In [ ]:
# -- Economy - Equities
DOMAIN203 = save_df(get_trend(DB_TREND, DB_TREND_ML, 'EQUITIES'), '203')
# DOMAIN203

# Save upper filter
UPPER_FILTER = DOMAIN203[['META', 'LABEL']].drop_duplicates()

UPPER_FILTER_ALL = UPPER_FILTER.copy()
UPPER_FILTER_ALL['META'] = ' All sectors'

UPPER_FILTER_203 = (pd.concat([UPPER_FILTER,
                               UPPER_FILTER_ALL],
                              axis=0).
                    sort_values(by=['META', 'LABEL'], ascending=True).
                    reset_index(drop=True))
bob.mongo.save_df(UPPER_FILTER_203, '203_UPPER_FILTER', DB_APP, True)
# UPPER_FILTER_203

In [ ]:
# -- Economy - Commodities
DOMAIN204 = save_df(get_trend(DB_TREND, DB_TREND_ML, 'COMMODITIES'), '204')
# DOMAIN204

# Save upper filter
UPPER_FILTER = DOMAIN204[['META', 'LABEL']].drop_duplicates()

UPPER_FILTER_ALL = UPPER_FILTER.copy()
UPPER_FILTER_ALL['META'] = ' All categories'

UPPER_FILTER_204 = (pd.concat([UPPER_FILTER,
                               UPPER_FILTER_ALL],
                              axis=0).
                    sort_values(by=['META', 'LABEL'], ascending=True).
                    reset_index(drop=True))
bob.mongo.save_df(UPPER_FILTER_204, '204_UPPER_FILTER', DB_APP, True)

In [ ]:
DONE = BHC.done(BHC_KEY)
print(DONE)